# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA (Low-Rank Adaptation)
* Model: GPT-2 (via Hugging Face – 'gpt2')
* Evaluation approach: Accuracy comparison before and after fine-tuning using Hugging Face `Trainer`
* Fine-tuning dataset: 'ag_news' dataset from Hugging Face (for news classification, simple and fast) 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Load dataset
dataset = load_dataset("ag_news")
dataset = dataset.map(lambda x: {"labels": x["label"]}, remove_columns=["label"])

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token


# Tokenize dataset
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

tokenized_ds = dataset.map(tokenize, batched=True)
tokenized_ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Load model for classification
model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=4)
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

# Define metric
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Training arguments
args = TrainingArguments(
    output_dir="/tmp/base_model",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    evaluation_strategy="epoch",
    logging_dir="/tmp/logs",
    logging_steps=10,
    save_total_limit=1
)

# Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds["train"].select(range(2000)),
    eval_dataset=tokenized_ds["test"].select(range(500)),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Evaluate base model
original_results = trainer.evaluate()
print("Original model performance:", original_results)


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Original model performance: {'eval_loss': 3.8886308670043945, 'eval_accuracy': 0.234, 'eval_runtime': 4.6224, 'eval_samples_per_second': 108.168, 'eval_steps_per_second': 27.042}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [ ]:
from peft import LoraConfig, get_peft_model

# Create LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn", "c_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

# Wrap base model in PEFT
peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

# Training arguments for PEFT
peft_args = TrainingArguments(
    output_dir="/tmp/peft_model",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="/tmp/peft_logs",
    logging_steps=10,
    load_best_model_at_end=True,
)

# PEFT Trainer
peft_trainer = Trainer(
    model=peft_model,
    args=peft_args,
    train_dataset=tokenized_ds["train"].select(range(2000)),
    eval_dataset=tokenized_ds["test"].select(range(500)),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train PEFT model
peft_trainer.train()

# Save PEFT adapter only
peft_model.save_pretrained("/tmp/peft_model")
tokenizer.save_pretrained("/tmp/peft_model")


###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [ ]:
# Saving the model
# model.save("/tmp/peft-model") - this is not working, already saved in previous step - check saved
import os

print(os.listdir("/tmp/peft_model"))


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
from peft import PeftModel, PeftConfig

# 1. Load PEFT config
peft_config = PeftConfig.from_pretrained("/tmp/peft_model")

# 2. Load tokenizer (with pad token fix)
tokenizer = AutoTokenizer.from_pretrained("/tmp/peft_model")
tokenizer.pad_token = tokenizer.eos_token

# 3. Load base model
base_model = AutoModelForSequenceClassification.from_pretrained(
    peft_config.base_model_name_or_path,
    num_labels=4
)
base_model.config.pad_token_id = tokenizer.pad_token_id
base_model.resize_token_embeddings(len(tokenizer))

# 4. Load adapter weights
peft_loaded = PeftModel.from_pretrained(base_model, "/tmp/peft_model")
peft_loaded.eval()

# 5. Re-evaluate using Trainer
peft_eval_trainer = Trainer(
    model=peft_loaded,
    args=args,
    eval_dataset=tokenized_ds["test"].select(range(500)),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 6. Evaluate and compare
peft_results = peft_eval_trainer.evaluate()
print("Fine-tuned PEFT model performance:", peft_results)

# 7. Compare with original
print("\n Comparison:")
print("Original:", original_results)
print("PEFT Fine-tuned:", peft_results)


In [ ]:

# 1. Try using the bitsandbytes package (installed in the workspace) to combine quantization and LoRA. This is also known as QLoRA

from transformers import BitsAndBytesConfig, AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig
import torch


# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load quantized model (4-bit GPT-2 with correct num_labels)
model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=4,
    quantization_config=bnb_config,
    device_map="auto"
)

model.config.pad_token_id = tokenizer.pad_token_id
model.resize_token_embeddings(len(tokenizer))

# LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn", "c_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

# Merge quantization + LoRA = QLoRA
qlora_model = get_peft_model(model, lora_config)
qlora_model.print_trainable_parameters()


In [ ]:
# 2. Try training the model using different PEFT configurations and compare the results

configs_to_try = [
    {"r": 4, "alpha": 8},
    {"r": 8, "alpha": 16},
    {"r": 16, "alpha": 32}
]

for cfg in configs_to_try:
    lora_config = LoraConfig(
        r=cfg["r"],
        lora_alpha=cfg["alpha"],
        target_modules=["c_attn", "c_proj"],
        lora_dropout=0.1,
        bias="none",
        task_type="SEQ_CLS"
    )
    
    model = get_peft_model(base_model, lora_config)
    model.print_trainable_parameters()

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_ds["train"].select(range(2000)),
        eval_dataset=tokenized_ds["test"].select(range(500)),
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    results = trainer.evaluate()
    print(f"Results for r={cfg['r']}, alpha={cfg['alpha']}: {results}")


In [ ]:
all_results = []

for cfg in configs_to_try:
    ...
    result = trainer.evaluate()
    all_results.append((cfg["r"], cfg["alpha"], result["eval_accuracy"]))

print("\nSummary of LoRA Config Results:")
for r, alpha, acc in all_results:
    print(f"r={r}, alpha={alpha} => Accuracy: {acc:.3f}")
